In [5]:
# 필요한 라이브러리 로드
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [9]:
# 데이터 로드
paths = os.listdir('../company_data')
company = pd.DataFrame()
for path in paths :
    df = pd.read_excel('../company_data/' + path)
    company = pd.concat([company, df])
company.sort_values(['거래소코드', '회계년도'], inplace=True)
company.reset_index(drop=True, inplace=True)

In [15]:
# 회계년도 12월 데이터만 남김
company = company.loc[company['회계년도'].str.endswith('/12'), : ]
company.reset_index(drop=True, inplace=True)
# 중간 저장
company.to_csv('../data/외감_1999_2024_origin.csv', index = False)

In [16]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533270 entries, 0 to 533269
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   회사명                    533270 non-null  object 
 1   거래소코드                  533270 non-null  int64  
 2   회계년도                   533270 non-null  object 
 3   산업코드                   533270 non-null  int64  
 4   산업명                    533270 non-null  object 
 5   종업원                    533270 non-null  int64  
 6   자산(*)(천원)              533129 non-null  float64
 7   비유동자산(*)(천원)           533129 non-null  float64
 8   유형자산(*)(천원)            533129 non-null  float64
 9   유동자산(*)(천원)            533129 non-null  float64
 10  재고자산(*)(천원)            533129 non-null  float64
 11  매출채권(*)(천원)            533129 non-null  float64
 12  자본(*)(천원)              533129 non-null  float64
 13  자본금(*)(천원)             533129 non-null  float64
 14  자본잉여금(*)(천원)           533129 non-nu

In [17]:
print(f"총 인덱스 수 : {len(company)}")
print(f"총 기업 수 : {len(company['거래소코드'].unique())}")

총 인덱스 수 : 533270
총 기업 수 : 73185


In [18]:
# 결측치 인덱스 제거
company = company.loc[company.isna().sum(axis=1) == 0 , : ]

In [19]:
print(f"결측치 제거 후 총 인덱스 수 : {len(company)}")
print(f"결측치 제거 후 총 기업 수 : {len(company['거래소코드'].unique())}")

결측치 제거 후 총 인덱스 수 : 533129
결측치 제거 후 총 기업 수 : 73176


In [22]:
# 금액 단위 통일
company.iloc[ : , 6 : ] = company.iloc[ : , 6 : ] * 1000
# 컬럼명 정리
company.columns = company.columns.str.replace('(*)', '')
company.columns = company.columns.str.replace('(천원)', '')
company.rename(columns ={
    '매출액(영업수익)' : '매출액',
    '판매비와관리비(영업비용)' : '판매비와관리비',
    '영업이익(손실)' : '영업이익',
    '법인세비용차감전(계속사업)손익' : '법인세비용차감전손익',
    '(계속사업손익)법인세비용' : '법인세비용',
    '당기순이익(순손실)' : '당기순이익'
},inplace=True)

In [ ]:
# 차입금 컬럼 생성
company['차입금'] = company['단기차입금'] + company['장기차입금']

In [44]:
# 중간저장
company.to_csv('../data/외감_1999_2024_nonnull.csv', index=False)

In [116]:
print(f"차입금 10억 이상 인덱스 수 : {len(company.loc[company['차입금'] >= 10**9 , : ])}")
print(f"차입금 10억 이상 기업 수 : {len(company.loc[company['차입금'] >= 10**9 , '거래소코드'].unique())}")
print(f"차입금 10억 이상 비율 : {len(company.loc[company['차입금'] >= 10**9 , : ])/len(company)}")
print(f"무차입 인덱스 수 : {len(company.loc[company['차입금'] == 0 , : ])}")
print(f"무차입 기업 수 : {len(company.loc[company['차입금'] == 0 , '거래소코드' ].unique())}")
print(f"무차입 비율 : {len(company.loc[company['차입금'] == 0 , : ])/len(company)}")

차입금 10억 이상 인덱스 수 : 413883
차입금 10억 이상 기업 수 : 64775
차입금 10억 이상 비율 : 0.7763280556863348
무차입 인덱스 수 : 81936
무차입 기업 수 : 20240
무차입 비율 : 0.15368888205293654


In [27]:
# 차입금이 10억 이상이 된 적 있는 기업만 남김
com_b = company.loc[company['차입금'] >= 10**9 , '거래소코드'].unique()
company_1b = company.loc[company['거래소코드'].isin(com_b), : ]

In [31]:
# 중간 저장
company_1b.to_csv('../data/외감_1999_2024_1b.csv', index=False)

In [36]:
company_1b.describe()

,거래소코드,산업코드,종업원,자산,비유동자산,유형자산,유동자산,재고자산,매출채권,자본,...,영업외비용,이자비용,법인세비용차감전손익,법인세비용,당기순이익,기초자본금,영업활동으로 인한 현금흐름,투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,차입금
count,492904.000000,492904.000000,492904.000000,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,...,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05,4.929040e+05
mean,55623.085982,71136.660074,80.033960,6.802874e+10,3.966700e+10,2.577833e+10,2.805225e+10,9.443594e+09,7.849384e+09,2.459173e+10,...,3.037636e+09,1.212135e+09,1.946465e+09,6.105004e+08,1.346819e+09,4.562844e+09,2.640092e+09,-4.294467e+09,1.963161e+09,1.897198e+10
std,28316.037926,43222.910088,410.276731,5.547906e+11,4.065130e+11,3.310784e+11,1.987576e+11,8.733768e+10,5.364897e+10,3.235346e+11,...,4.002794e+10,1.185972e+10,5.380432e+10,6.966709e+09,5.083957e+10,4.739172e+10,5.514553e+10,5.283973e+10,4.078761e+10,8.980656e+10
min,10000.000000,10100.000000,0.000000,-4.061700e+07,-1.964740e+08,-4.939030e+08,-4.807000e+07,-3.006420e+08,-2.461890e+08,-2.183369e+13,...,-1.027705e+10,-3.806243e+09,-2.023659e+13,-9.734610e+11,-2.023659e+13,0.000000e+00,-2.677199e+12,-9.286154e+12,-5.965759e+12,-6.719625e+09
25%,30251.000000,32602.000000,0.000000,1.311905e+10,4.485403e+09,1.172289e+09,4.025604e+09,1.332500e+07,1.622370e+08,1.636322e+09,...,2.556065e+08,1.096442e+08,-3.707760e+08,0.000000e+00,-3.855512e+08,1.000000e+08,-5.790992e+08,-2.814261e+09,-9.791775e+08,2.500000e+09
50%,53730.000000,64101.000000,16.000000,2.085546e+10,1.000000e+10,6.766594e+09,9.042431e+09,1.241154e+09,1.916132e+09,5.919046e+09,...,5.737920e+08,3.119150e+08,3.774010e+08,3.246750e+07,3.100250e+08,6.000000e+08,5.693700e+08,-5.959420e+08,0.000000e+00,6.742063e+09
75%,76817.000000,105901.000000,65.000000,4.224442e+10,2.099254e+10,1.467793e+10,1.940317e+10,5.145294e+09,5.445514e+09,1.486900e+10,...,1.454892e+09,7.184935e+08,1.845589e+09,3.163315e+08,1.521512e+09,2.000000e+09,2.519309e+09,0.000000e+00,2.093903e+09,1.420391e+10
max,116714.000000,209801.000000,32969.000000,7.250836e+13,6.172020e+13,5.630496e+13,3.709545e+13,1.112437e+13,7.685215e+12,5.923572e+13,...,1.343886e+13,2.876034e+12,1.668343e+13,1.056607e+12,1.668343e+13,7.688422e+12,1.038106e+13,2.273928e+12,5.521992e+12,1.521693e+13


In [37]:
# 값이 음수면 안되는데 음수인 값을 가진 데이터 삭제
cols = ['판매비와관리비', '매출원가', '매출액', '자산', '유동자산', '비유동자산', '유형자산', '재고자산', '부채', '비유동부채', '장기차입금', '단기차입금', '유동부채', '감가상각비', '영업외수익', '영업외비용', '이자비용', '자본금', '기초자본금', '종업원']
company_clean = company_1b.copy()
for col in cols :
    company_clean = company_clean.loc[company_clean[col] >= 0 , : ]

In [ ]:
# 기초통계량 확인
company_clean.describe()

,거래소코드,산업코드,종업원,자산,비유동자산,유형자산,유동자산,재고자산,매출채권,자본,...,영업외비용,이자비용,법인세비용차감전손익,법인세비용,당기순이익,기초자본금,영업활동으로 인한 현금흐름,투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,차입금
count,491920.000000,491920.000000,491920.000000,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,...,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05
mean,55603.415761,71086.966915,80.151567,6.806185e+10,3.969234e+10,2.581377e+10,2.805942e+10,9.438961e+09,7.856291e+09,2.462304e+10,...,3.038005e+09,1.211917e+09,1.952146e+09,6.113276e+08,1.351719e+09,4.563113e+09,2.643657e+09,-4.302193e+09,1.969884e+09,1.896140e+10
std,28317.517370,43213.760183,410.658713,5.553349e+11,4.069125e+11,3.314054e+11,1.989438e+11,8.741072e+10,5.369591e+10,3.238495e+11,...,4.006561e+10,1.187102e+10,5.385132e+10,6.972894e+09,5.088368e+10,4.743271e+10,5.519608e+10,5.288985e+10,4.082009e+10,8.988313e+10
min,10000.000000,10100.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.461890e+08,-2.183369e+13,...,0.000000e+00,0.000000e+00,-2.023659e+13,-9.734610e+11,-2.023659e+13,0.000000e+00,-2.677199e+12,-9.286154e+12,-5.965759e+12,0.000000e+00
25%,30231.000000,32602.000000,0.000000,1.311798e+10,4.494332e+09,1.187569e+09,4.023844e+09,1.366750e+07,1.641430e+08,1.641687e+09,...,2.556738e+08,1.097118e+08,-3.681982e+08,0.000000e+00,-3.831720e+08,1.000000e+08,-5.779055e+08,-2.818165e+09,-9.748942e+08,2.500000e+09
50%,53704.000000,64101.000000,16.000000,2.084630e+10,1.000291e+10,6.779048e+09,9.039253e+09,1.241052e+09,1.920152e+09,5.927138e+09,...,5.735455e+08,3.117700e+08,3.789110e+08,3.275500e+07,3.111360e+08,6.000000e+08,5.694860e+08,-5.977205e+08,0.000000e+00,6.737744e+09
75%,76795.000000,105901.000000,65.000000,4.220465e+10,2.097552e+10,1.469187e+10,1.939401e+10,5.137196e+09,5.451628e+09,1.487932e+10,...,1.453225e+09,7.172765e+08,1.847269e+09,3.168212e+08,1.522754e+09,2.000000e+09,2.516802e+09,0.000000e+00,2.095826e+09,1.418720e+10
max,116714.000000,209801.000000,32969.000000,7.250836e+13,6.172020e+13,5.630496e+13,3.709545e+13,1.112437e+13,7.685215e+12,5.923572e+13,...,1.343886e+13,2.876034e+12,1.668343e+13,1.056607e+12,1.668343e+13,7.688422e+12,1.038106e+13,2.273928e+12,5.521992e+12,1.521693e+13


In [42]:
# 중간 저장
company_clean.to_csv('../data/외감_1999_2024_clean.csv', index=False)

In [48]:
# 1% 윈저라이징
company_wz = company_clean.copy()
# 1% 윈저라이징
for col in company_wz.columns[5 : ] :
    company_wz.loc[company_wz[col] >= company_wz[col].quantile(0.995) , col] = company_wz[col].quantile(0.995)
    company_wz.loc[company_wz[col] <= company_wz[col].quantile(0.005), col] = company_wz[col].quantile(0.005)

In [49]:
# 기초통계량 확인
company_wz.describe()

,거래소코드,산업코드,종업원,자산,비유동자산,유형자산,유동자산,재고자산,매출채권,자본,...,영업외비용,이자비용,법인세비용차감전손익,법인세비용,당기순이익,기초자본금,영업활동으로 인한 현금흐름,투자활동으로 인한 현금흐름,재무활동으로 인한 현금흐름,차입금
count,491920.000000,491920.000000,491920.000000,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,...,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05
mean,55603.415761,71086.966915,67.435514,5.327610e+10,2.963652e+10,1.824333e+10,2.280968e+10,7.370017e+09,6.326434e+09,1.840139e+10,...,2.141569e+09,9.703910e+08,1.510186e+09,4.851835e+08,1.034731e+09,3.457241e+09,1.789369e+09,-3.252369e+09,1.729045e+09,1.663831e+10
std,28317.517370,43213.760183,167.216201,1.234082e+11,8.448676e+10,4.947624e+10,5.099494e+10,2.167082e+10,1.615065e+10,5.761720e+10,...,6.337740e+09,2.670882e+09,9.626174e+09,1.762200e+09,8.277926e+09,1.244202e+10,1.355621e+10,1.250028e+10,1.516326e+10,3.852447e+10
min,10000.000000,10100.000000,0.000000,2.006185e+09,0.000000e+00,0.000000e+00,2.710257e+07,0.000000e+00,0.000000e+00,-7.272016e+10,...,0.000000e+00,0.000000e+00,-3.840165e+10,-2.133931e+09,-3.820542e+10,0.000000e+00,-6.358451e+10,-1.179912e+11,-7.380364e+10,0.000000e+00
25%,30231.000000,32602.000000,0.000000,1.311798e+10,4.494332e+09,1.187569e+09,4.023844e+09,1.366750e+07,1.641430e+08,1.641687e+09,...,2.556738e+08,1.097118e+08,-3.681982e+08,0.000000e+00,-3.831720e+08,1.000000e+08,-5.779055e+08,-2.818165e+09,-9.748942e+08,2.500000e+09
50%,53704.000000,64101.000000,16.000000,2.084630e+10,1.000291e+10,6.779048e+09,9.039253e+09,1.241052e+09,1.920152e+09,5.927138e+09,...,5.735455e+08,3.117700e+08,3.789110e+08,3.275500e+07,3.111360e+08,6.000000e+08,5.694860e+08,-5.977205e+08,0.000000e+00,6.737744e+09
75%,76795.000000,105901.000000,65.000000,4.220465e+10,2.097552e+10,1.469187e+10,1.939401e+10,5.137196e+09,5.451628e+09,1.487932e+10,...,1.453225e+09,7.172765e+08,1.847269e+09,3.168212e+08,1.522754e+09,2.000000e+09,2.516802e+09,0.000000e+00,2.095826e+09,1.418720e+10
max,116714.000000,209801.000000,1549.000000,1.203297e+12,8.477923e+11,4.935231e+11,4.668393e+11,1.995381e+11,1.501288e+11,5.639710e+11,...,6.312173e+10,2.637651e+10,8.248089e+10,1.743996e+10,6.736717e+10,1.250185e+11,1.103851e+11,2.755370e+10,1.185616e+11,3.592405e+11


In [50]:
# 중간저장
company_wz.to_csv('../data/외감_1999_2024_wz.csv', index=False)

In [88]:
gaap = company_wz.copy()

In [89]:
gaap.reset_index(drop=True,inplace=True)

In [90]:
# 년도 컬럼 생성
gaap.insert(3, '년도', gaap['회계년도'].map(lambda x : int(x[ : 4])))

In [91]:
# 파생변수 생성
gaap['매출총이익률'] = ((gaap['매출액'] - gaap['매출원가'])/(gaap['매출액'])) * 100
gaap['영업이익률'] = ((gaap['영업이익'])/(gaap['매출액'])) * 100
gaap['순이익률'] = ((gaap['당기순이익'])/(gaap['매출액'])) * 100
gaap['부채비율'] = ((gaap['부채'])/(gaap['자본'])) * 100
gaap['유동부채비율'] = ((gaap['유동부채'])/(gaap['자본'])) * 100
gaap['비유동부채비율'] = ((gaap['비유동부채'])/(gaap['자본'])) * 100
gaap['유동비율'] = ((gaap['유동자산'])/(gaap['유동부채'])) * 100
gaap['이자보상배율'] = (gaap['영업이익'])/(gaap['이자비용'])
gaap['금융비용부담률'] = ((gaap['이자비용'])/(gaap['매출액'])) * 100
gaap['자본유보율'] = ((gaap['자본잉여금'] + gaap['이익잉여금'])/(gaap['자본금'])) * 100
gaap['자본잠식률'] = ((gaap['자본금'] - gaap['자본'])/gaap['자본금']) * 100
gaap['부채 대비 영업활동으로 인한 현금흐름'] = gaap['영업활동으로 인한 현금흐름']/gaap['부채']
gaap['부채 대비 투자활동으로 인한 현금흐름'] = gaap['투자활동으로 인한 현금흐름']/gaap['부채']
gaap['부채 대비 재무활동으로 인한 현금흐름'] = gaap['재무활동으로 인한 현금흐름']/gaap['부채']

In [96]:
# 파생변수 중 음/양 무한대 값을 무한대 제외한 min/max로 치환
for col in gaap.columns[39 : ] :
    gaap.loc[gaap[col].isin([np.inf]), col] = gaap.loc[~gaap[col].isin([np.inf, -np.inf]), col].max()
    gaap.loc[gaap[col].isin([-np.inf]), col] = gaap.loc[~gaap[col].isin([np.inf, -np.inf]), col].min()

# 분모, 분자가 전부 0이어서 nan이 된 파생변수를 0으로 치환
gaap.fillna(0, inplace=True)

In [98]:
# 동적 파생변수 생성
E = (gaap['자본'] + gaap['자본'].shift()) * 0.5
A = (gaap['자산'] + gaap['자산'].shift()) * 0.5
IC = ((gaap['유형자산'] + gaap['유동자산'] - gaap['유동부채']) + (gaap['유형자산'] + gaap['유동자산'] - gaap['유동부채']).shift()) * 0.5
WC = ((gaap['유동자산'] - gaap['유동부채']) + (gaap['유동자산'] - gaap['유동부채']).shift()) * 0.5
P = (gaap['유형자산'] + gaap['유형자산'].shift()) * 0.5
T = (gaap['매출채권'] + gaap['매출채권'].shift()) * 0.5

gaap['ROE'] = (gaap['당기순이익']/E) * 100
gaap['ROA'] = (gaap['당기순이익']/A) * 100
gaap['ROIC'] = ((gaap['법인세비용차감전손익'] - gaap['법인세비용'])/IC) * 100
gaap['매출액증가율'] = ((gaap['매출액'].diff())/(gaap['매출액'].shift())) * 100
gaap['영업이익증가율'] = (gaap['영업이익'].diff())/(gaap['영업이익'].shift()) * 100
gaap['순이익증가율'] = (gaap['당기순이익'].diff())/(gaap['당기순이익'].shift()) * 100
gaap['총자산증가율'] = (gaap['자산'].diff())/(gaap['자산'].shift()) * 100
gaap['유동자산증가율'] = (gaap['유동자산'].diff())/(gaap['유동자산'].shift()) * 100
gaap['유형자산증가율'] = (gaap['유형자산'].diff())/(gaap['유형자산'].shift()) * 100
gaap['자기자본증가율'] = (gaap['자본'].diff())/(gaap['자본'].shift()) * 100
gaap['총자산회전율'] = (gaap['매출액']/A) * 100
gaap['자기자본회전율'] = (gaap['매출액']/E) * 100
gaap['순운전자본회전율'] = (gaap['매출액']/WC) * 100
gaap['유형자산회전율'] = (gaap['매출액']/P) * 100
gaap['매출채권회전율'] = (gaap['매출액']/T) * 100

# 전 인덱스와 연속되지 않는 컬럼 정리
flag = (gaap['거래소코드'] != gaap['거래소코드'].shift()) | ((gaap['년도'] - 1) != gaap['년도'].shift())

gaap.loc[flag, ['매출액증가율', '영업이익증가율', '순이익증가율', '총자산증가율', '유동자산증가율', '유형자산증가율', '자기자본증가율']] = 0

# 전기 당기 평균값이 필요한데 전기값이 없는 인덱스는 당기로 대체
gaap.loc[flag, 'ROE'] = (gaap.loc[flag, '당기순이익']/gaap.loc[flag, '자본']) * 100
gaap.loc[flag, 'ROA'] = (gaap.loc[flag, '당기순이익']/gaap.loc[flag, '자산']) * 100
gaap.loc[flag, 'ROIC'] = ((gaap.loc[flag, '법인세비용차감전손익'] - gaap.loc[flag, '법인세비용'])/(gaap.loc[flag, '유형자산'] + gaap.loc[flag, '유동자산'] - gaap.loc[flag, '유동부채'])) * 100
gaap.loc[flag, '총자산회전율'] = (gaap.loc[flag, '매출액']/gaap.loc[flag, '자산']) * 100
gaap.loc[flag, '자기자본회전율'] = (gaap.loc[flag, '매출액']/gaap.loc[flag, '자본']) * 100
gaap.loc[flag, '순운전자본회전율'] = (gaap.loc[flag, '매출액']/(gaap.loc[flag, '유동자산'] - gaap.loc[flag, '유동부채'])) * 100
gaap.loc[flag, '유형자산회전율'] = (gaap.loc[flag,'매출액']/gaap.loc[flag, '유형자산']) * 100
gaap.loc[flag, '매출채권회전율'] = (gaap.loc[flag, '매출액']/gaap.loc[flag, '매출채권']) * 100

In [100]:
# 파생변수 중 음/양 무한대 값을 무한대 제외한 min/max로 치환
for col in gaap.columns[53 : ] :
    gaap.loc[gaap[col].isin([np.inf]), col] = gaap.loc[~gaap[col].isin([np.inf, -np.inf]), col].max()
    gaap.loc[gaap[col].isin([-np.inf]), col] = gaap.loc[~gaap[col].isin([np.inf, -np.inf]), col].min()

# 분모, 분자가 전부 0이어서 nan이 된 파생변수를 0으로 치환
gaap.fillna(0, inplace=True)

In [101]:
# Beneish M-Score 생성을 위한 파생변수
gaap['DSRI'] = (gaap['매출채권']/gaap['매출액'])/((gaap['매출채권']/gaap['매출액']).shift())
gaap['GMI']= gaap['매출총이익률']/(gaap['매출총이익률'].shift())
gaap['AQI']= (1 - ((gaap['유동자산'] + gaap['유형자산'])/gaap['자산']))/((1 - ((gaap['유동자산'] + gaap['유형자산'])/gaap['자산'])).shift())
gaap['SGI']= gaap['매출액']/(gaap['매출액'].shift())
gaap['DEPI'] = (gaap['감가상각비']/(gaap['감가상각비'] + gaap['유형자산']))/(gaap['감가상각비']/((gaap['감가상각비'] + gaap['유형자산']).shift()))
gaap['SGAI'] = (gaap['판매비와관리비']/gaap['매출액'])/((gaap['판매비와관리비']/gaap['매출액']).shift())
gaap['LVGI'] = (gaap['부채']/gaap['자산'])/((gaap['부채']/gaap['자산']).shift())
gaap['TATA'] = (gaap['당기순이익'] - gaap['영업활동으로 인한 현금흐름'])/gaap['자산']

# 전 인덱스와 연속되지 않는 컬럼 정리
flag = (gaap['거래소코드'] != gaap['거래소코드'].shift()) | ((gaap['년도'] - 1) != gaap['년도'].shift())
gaap.loc[flag, ['DSRI', 'GMI', 'AQI', 'SGI', 'DEPI', 'SGAI', 'LVGI']] = 0

In [103]:
# 파생변수 중 음/양 무한대 값을 무한대 제외한 min/max로 치환
for col in gaap.columns[68 : ] :
    gaap.loc[gaap[col].isin([np.inf]), col] = gaap.loc[~gaap[col].isin([np.inf, -np.inf]), col].max()
    gaap.loc[gaap[col].isin([-np.inf]), col] = gaap.loc[~gaap[col].isin([np.inf, -np.inf]), col].min()

# 분모, 분자가 전부 0이어서 nan이 된 파생변수를 0으로 치환
gaap.fillna(0, inplace=True)

In [105]:
#Beneish M-Score 생성
gaap['Beneish M-Score'] = -4.84 + 0.92*gaap['DSRI']+0.528*gaap['GMI']+0.404*gaap['AQI']+0.892*gaap['SGI']+0.115*gaap['DEPI']-0.172*gaap['SGAI']+4.679*gaap['TATA']-0.327*gaap['LVGI']

In [106]:
# 파생변수 기초통계량 확인
gaap.iloc[ : , 39 : ].describe()

,매출총이익률,영업이익률,순이익률,부채비율,유동부채비율,비유동부채비율,유동비율,이자보상배율,금융비용부담률,자본유보율,...,매출채권회전율,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,Beneish M-Score
count,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,...,4.919200e+05,4.919200e+05,4.919200e+05,4.919200e+05,491920.000000,4.919200e+05,491920.000000,491920.000000,491920.000000,4.919200e+05
mean,-1.052667e+05,-1.774885e+07,-1.329263e+07,1.132004e+05,1.070056e+05,1.581784e+04,4.954581e+02,1.399005e+06,2.150002e+06,1.604911e+03,...,1.102572e+09,2.747485e+05,-9.460841e+05,3.079972e+06,8576.452851,1.050867e+02,3883.113272,0.919356,-0.043550,1.004534e+06
std,3.887458e+06,7.039800e+07,5.606650e+07,1.699863e+07,1.664713e+07,5.874985e+06,7.086557e+03,8.019585e+06,1.001332e+07,1.922509e+04,...,3.440157e+09,2.097675e+06,5.608082e+07,3.818487e+07,66199.591529,5.395732e+04,47573.579076,3.304308,0.644224,3.334573e+07
min,-1.437468e+08,-2.969817e+08,-2.492698e+08,-1.922670e+08,-1.147431e+08,-1.216624e+08,6.508613e-03,-6.561700e+06,0.000000e+00,-1.391953e+06,...,0.000000e+00,0.000000e+00,-1.320244e+09,-2.401654e+08,0.000000,0.000000e+00,0.000000,0.000000,-74.066206,-7.941159e+08
25%,8.846945e+00,-3.023299e+00,-6.269179e+00,4.017773e+01,2.218228e+01,1.618519e+00,5.876094e+01,-3.759508e-01,4.965185e-01,-1.257764e+01,...,4.044465e+02,0.000000e+00,3.705718e-01,5.335817e-01,0.610488,3.612971e-01,0.518311,0.822494,-0.075773,-3.644296e+00
50%,1.818286e+01,3.491522e+00,1.779078e+00,1.444911e+02,8.738669e+01,2.145243e+01,1.063112e+02,1.556410e+00,1.919684e+00,2.814954e+02,...,7.325160e+02,8.557042e-01,9.560186e-01,9.545153e-01,0.989855,9.803878e-01,0.949283,0.976596,-0.017492,-2.610950e+00
75%,3.807421e+01,9.629862e+00,6.867146e+00,3.518905e+02,2.190835e+02,9.021079e+01,1.842201e+02,6.651556e+00,7.158256e+00,1.312782e+03,...,1.861137e+03,1.152258e+00,1.074159e+00,1.185389e+00,1.172575,1.040270e+00,1.132618,1.044856,0.035146,-1.834612e+00
max,1.000000e+02,1.000000e+02,3.878590e+06,2.687291e+09,2.687291e+09,2.367920e+09,1.088521e+06,3.979422e+07,4.879191e+07,1.862835e+06,...,1.183712e+10,1.629086e+07,9.125594e+07,3.935696e+08,519897.563333,3.775333e+07,587467.039234,2032.193376,33.599249,2.221728e+08


In [107]:
# 중간 저장
gaap.to_csv('../data/외감_1999_2024_파생변수.csv', index=False)

In [108]:
gaap.reset_index(drop=True,inplace=True)

In [109]:
# 라벨링
gaap['label'] = ((gaap['이자보상배율'].shift(-1) < 1) | (gaap['자본잠식률'].shift(-1) >= 50)).astype(int)

# 다음해 데이터가 없는 경우 정상으로 분류
gaap.loc[gaap['거래소코드'] != gaap['거래소코드'].shift(-1), 'label'] = 0
gaap.loc[(gaap['년도'] + 1) != gaap['년도'].shift(-1), 'label' ] = 0

# 전 해 데이터가 없는데 이자보상배율이 1 미만이거나 자본잠식률이 50% 이상인 경우 부실로 분류
flag1 = (gaap['거래소코드'] != gaap['거래소코드'].shift()) | ((gaap['년도'] - 1) != gaap['년도'].shift())
flag2 = (gaap['이자보상배율'] < 1) | (gaap['자본잠식률'] >= 50)
gaap.loc[flag1 & flag2, 'label'] = 1

# 한번 부실 징후가 나타난 기업은 이후 데이터 삭제
gaap['target'] = np.nan
gaap.loc[gaap['거래소코드'] != gaap['거래소코드'].shift(), 'target'] = 0
gaap.loc[(gaap['label'].shift() == 1) & (gaap['거래소코드'] == gaap['거래소코드'].shift()), 'target'] = '삭제'
gaap['target'].fillna(method = 'ffill', inplace=True)

gaap = gaap.loc[gaap['target'] != '삭제' , : ]

# 1999년, 2024년 데이터 삭제
gaap = gaap.loc[~gaap['년도'].isin([1999, 2024]), : ]

# 상장 데이터와 겹치는 기업 삭제
market = pd.read_excel('../data/상장_gaap_ifrs_ratios(1999-2023).xlsx')
gaap = gaap.loc[~gaap['거래소코드'].isin([market['거래소코드'].unique()]), : ]

# 불필요한 컬럼 삭제
gaap.drop(['년도', 'target'], axis=1, inplace=True)

# 인덱스 리셋
gaap.reset_index(drop=True, inplace=True)

In [111]:
# 데이터 확인
gaap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168570 entries, 0 to 168569
Data columns (total 77 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   회사명                   168570 non-null  object 
 1   거래소코드                 168570 non-null  int64  
 2   회계년도                  168570 non-null  object 
 3   산업코드                  168570 non-null  int64  
 4   산업명                   168570 non-null  object 
 5   종업원                   168570 non-null  int64  
 6   자산                    168570 non-null  float64
 7   비유동자산                 168570 non-null  float64
 8   유형자산                  168570 non-null  float64
 9   유동자산                  168570 non-null  float64
 10  재고자산                  168570 non-null  float64
 11  매출채권                  168570 non-null  float64
 12  자본                    168570 non-null  float64
 13  자본금                   168570 non-null  float64
 14  자본잉여금                 168570 non-null  float64
 15  

In [117]:
print(f"총 인덱스 수 : {len(gaap)}")
print(f"총 기업 수 : {len(gaap['거래소코드'].unique())}")
print(f"부실 징후 인덱스 수 : {len(gaap.loc[gaap['label'] == 1, : ])}")
print(f"부실 기업 수 : {len(gaap.loc[gaap['label'] == 1, '거래소코드'].unique())}")
print(f"정상 인덱스 수 : {len(gaap.loc[gaap['label'] == 0, : ])}")
print(f"정상 기업 수 : {len(gaap.loc[gaap['label'] == 0, '거래소코드'].unique())}")

총 인덱스 수 : 168570
총 기업 수 : 61724
부실 징후 인덱스 수 : 50319
부실 기업 수 : 50319
정상 인덱스 수 : 118251
정상 기업 수 : 24718


In [113]:
# 라벨별 확인
gaap_1 = gaap.loc[gaap['label'] == 1, : ]
gaap_0 = gaap.loc[gaap['label'] == 0, : ]

In [114]:
gaap_1.describe()

,거래소코드,산업코드,종업원,자산,비유동자산,유형자산,유동자산,재고자산,매출채권,자본,...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,Beneish M-Score,label
count,50319.000000,50319.000000,50319.000000,5.031900e+04,5.031900e+04,5.031900e+04,5.031900e+04,5.031900e+04,5.031900e+04,5.031900e+04,...,5.031900e+04,50319.000000,5.031900e+04,50319.000000,50319.000000,50319.000000,50319.000000,50319.000000,5.031900e+04,50319.0
mean,70184.260299,80773.211411,33.258868,3.101983e+10,1.493353e+10,9.983389e+09,1.581623e+10,6.798968e+09,2.894879e+09,7.633907e+09,...,5.406890e+04,0.264500,3.749898e+05,0.348611,0.476005,0.409138,0.260749,0.063002,2.012351e+05,1.0
std,27335.472520,44431.265183,110.297594,7.460903e+10,5.010539e+10,3.019806e+10,3.531596e+10,1.848123e+10,9.977767e+09,3.390943e+10,...,9.369548e+05,0.498388,1.275098e+07,5.965829,20.005753,15.136946,0.520029,0.413637,5.275715e+06,0.0
min,10000.000000,10100.000000,0.000000,2.006185e+09,0.000000e+00,0.000000e+00,2.710257e+07,0.000000e+00,0.000000e+00,-7.272016e+10,...,0.000000e+00,0.000000,-2.401654e+08,0.000000,0.000000,0.000000,0.000000,-37.389596,-9.702684e+07,1.0
25%,49260.500000,32900.000000,0.000000,1.065640e+10,1.234608e+09,7.198050e+07,2.706866e+09,0.000000e+00,0.000000e+00,2.955000e+05,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-0.061846,-4.980679e+00,1.0
50%,70367.000000,74604.000000,2.000000,1.513236e+10,6.761827e+09,4.152089e+09,7.268117e+09,8.999520e+08,4.536230e+08,1.972271e+09,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000485,-4.495278e+00,1.0
75%,93153.000000,126801.000000,26.000000,2.582807e+10,1.276999e+10,1.002324e+10,1.498764e+10,5.833299e+09,2.471738e+09,6.225244e+09,...,0.000000e+00,0.546393,2.404427e-01,0.481254,0.000000,0.518753,0.552744,0.115645,-2.488931e+00,1.0
max,116714.000000,209801.000000,1549.000000,1.203297e+12,8.477923e+11,4.935231e+11,4.668393e+11,1.995381e+11,1.501288e+11,5.639710e+11,...,1.629086e+07,11.610125,3.935696e+08,1263.518889,2832.093886,3086.366690,28.559662,9.654529,1.739897e+08,1.0


In [115]:
gaap_0.describe()

,거래소코드,산업코드,종업원,자산,비유동자산,유형자산,유동자산,재고자산,매출채권,자본,...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,Beneish M-Score,label
count,118251.000000,118251.000000,118251.000000,1.182510e+05,1.182510e+05,1.182510e+05,1.182510e+05,1.182510e+05,1.182510e+05,1.182510e+05,...,1.182510e+05,118251.000000,1.182510e+05,118251.000000,118251.000000,118251.000000,118251.000000,118251.000000,1.182510e+05,118251.0
mean,56218.232818,59978.765144,93.867612,4.657806e+10,2.389637e+10,1.475619e+10,2.238066e+10,5.323904e+09,8.095006e+09,2.331981e+10,...,9.519762e+04,0.828870,8.826497e+05,1.040211,1.297953,0.903109,0.792546,-0.004627,4.441685e+05,0.0
std,27872.789971,35884.790917,204.050793,1.172984e+11,7.682009e+10,4.366307e+10,4.882912e+10,1.633564e+10,1.708004e+10,6.053903e+10,...,1.241680e+06,0.606517,2.064865e+07,10.000292,75.596353,8.502271,0.680220,0.179280,8.463953e+06,0.0
min,10000.000000,10100.000000,0.000000,2.006185e+09,0.000000e+00,0.000000e+00,2.710257e+07,0.000000e+00,0.000000e+00,6.884000e+06,...,0.000000e+00,0.000000,-2.401654e+08,0.000000,0.000000,0.000000,0.000000,-13.162994,-9.702684e+07,0.0
25%,38803.000000,32501.000000,4.000000,1.306837e+10,4.464670e+09,2.112928e+09,5.763930e+09,1.643715e+08,1.455292e+09,4.351048e+09,...,3.194626e-01,0.709041,4.716373e-01,0.755246,0.349392,0.680303,0.694864,-0.060328,-3.335491e+00,0.0
50%,56854.000000,33101.000000,37.000000,1.925680e+10,8.598749e+09,6.145756e+09,1.027975e+10,1.421343e+09,3.497382e+09,8.744094e+09,...,9.053552e-01,0.960959,9.257819e-01,1.017007,0.940588,0.958236,0.933988,-0.009234,-2.546186e+00,0.0
75%,77222.000000,74607.000000,91.000000,3.492062e+10,1.622014e+10,1.209429e+10,1.919322e+10,4.378386e+09,7.479221e+09,1.784040e+10,...,1.131986e+00,1.065806,1.170117e+00,1.173167,1.024620,1.099710,1.010500,0.048684,-1.978787e+00,0.0
max,116652.000000,209801.000000,1549.000000,1.203297e+12,8.477923e+11,4.935231e+11,4.668393e+11,1.995381e+11,1.501288e+11,5.639710e+11,...,1.629086e+07,100.399319,3.935696e+08,2895.052858,24769.193634,2722.265514,90.947590,3.006409,1.739897e+08,0.0


In [118]:
# 최종 데이터 저장
gaap.to_csv('../외감_2000_2023_label_최종.csv', index=False)